### Imports

In [1]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt
import os
import pathlib
from utils import read_data
import random
import numpy as np

In [2]:
def split_data(ds, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=699):
    assert (train_split + test_split + val_split) == 1
    
    ds_size = len(ds)
    if shuffle:
        random.shuffle(ds)
        
    train_ds = ds[:int(ds_size * train_split)]
    val_ds = ds[int(ds_size * train_split):int(ds_size * train_split + ds_size * val_split)]
    test_ds = ds[int(ds_size * train_split + ds_size * val_split):]
    
    return train_ds, test_ds, val_ds

In [3]:
dataset = read_data(color=True)
train_ds, test_ds, val_ds = split_data(dataset)

In [4]:
vgg16 = tf.keras.applications.VGG16(
    include_top=False,
    weights="imagenet")

vgg16.trainable = False

inputs = tf.keras.Input(shape=(224, 224, 3))
x = tf.keras.applications.vgg16.preprocess_input(
    inputs, data_format=None)

x = vgg16(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(4096, activation="relu")(x)
x = tf.keras.layers.Dense(4096, activation="relu")(x)
outputs = tf.keras.layers.Dense(2, activation="sigmoid")(x)
model = tf.keras.Model(inputs, outputs)
model.summary()

2022-04-18 18:50:06.224203: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-18 18:50:07.281250: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/miglou/.local/lib/python3.8/site-packages/cv2/../../lib64:
2022-04-18 18:50:07.281305: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-04-18 18:50:07.319131: I tensorflow/core/platform/cpu_feature_guard.cc:151] 

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 tf.__operators__.getitem (S  (None, 224, 224, 3)      0         
 licingOpLambda)                                                 
                                                                 
 tf.nn.bias_add (TFOpLambda)  (None, 224, 224, 3)      0         
                                                                 
 vgg16 (Functional)          (None, None, None, 512)   14714688  
                                                                 
 global_average_pooling2d (G  (None, 512)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 512)               0     

In [5]:
from tensorflow.keras.optimizers import Adam
opt = Adam(learning_rate=0.0001)
model.compile(optimizer=opt, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False), metrics=['accuracy'])

history = model.fit(np.array([x[0] for x in train_ds]), np.array([x[1] for x in train_ds]),
                    epochs=10, validation_data=(np.array([x[0] for x in val_ds]), np.array([x[1] for x in val_ds])))

Epoch 1/10
7/7 [==============================] - 28s 4s/step - loss: 1.3201 - accuracy: 0.4505 - val_loss: 0.7204 - val_accuracy: 0.5600
Epoch 2/10
7/7 [==============================] - 25s 4s/step - loss: 0.9709 - accuracy: 0.5495 - val_loss: 0.8891 - val_accuracy: 0.4400
Epoch 3/10
7/7 [==============================] - 26s 4s/step - loss: 0.8633 - accuracy: 0.5198 - val_loss: 0.6968 - val_accuracy: 0.5600
Epoch 4/10
1/7 [===>..........................] - ETA: 21s - loss: 0.7698 - accuracy: 0.5625

KeyboardInterrupt: 

In [ ]:
model.evaluate(np.array([x[0] for x in test_ds]), np.array([x[1] for x in test_ds]))